In [ ]:
!apt update && apt install -y python3.9 python3.9-distutils
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 1
!update-alternatives --config python3

In [ ]:
!apt-get update
!apt-get install python3.9-distutils python3.9-venv  # Ensure Python 3.9 has distutils
!wget https://bootstrap.pypa.io/get-pip.py
!python3.9 get-pip.py --force-reinstall

In [ ]:
!python3 --version

In [ ]:
!pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116
!pip install "numpy<2" basicsr==1.4.2 gfpgan==1.3.8 facexlib opencv-python

In [ ]:
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd /content/Real-ESRGAN
!pip install -e .

In [ ]:
!mkdir -p weights
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P weights

In [ ]:
from google.colab import files
import os

uploaded = files.upload()
input_video = list(uploaded.keys())[0]

In [ ]:
import cv2

frames_dir = 'input_frames'
os.makedirs(frames_dir, exist_ok=True)

cap = cv2.VideoCapture(input_video)
fps = cap.get(cv2.CAP_PROP_FPS)  # Get original FPS

i = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    cv2.imwrite(f'{frames_dir}/frame_{i:04d}.png', frame)
    i += 1
cap.release()

print(f"Extracted {i} frames to {frames_dir}")

In [ ]:
output_frames = 'output_frames'
os.makedirs(output_frames, exist_ok=True)

!python inference_realesrgan.py \
    -n RealESRGAN_x4plus \
    -i {frames_dir} \
    -o {output_frames} \
    -s 4 \
    --face_enhance \
    --model_path weights/RealESRGAN_x4plus.pth


In [ ]:
import cv2
import os

output_video = 'enhanced_video.mp4'
frames = sorted([f for f in os.listdir(output_frames) if f.endswith('.png')])

if not frames:
    raise ValueError("No output frames found! Check processing step.")

# Get dimensions from first frame
frame_path = os.path.join(output_frames, frames[0])
h, w = cv2.imread(frame_path).shape[:2]

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, fps, (w, h))

for frame_name in frames:
    frame = cv2.imread(os.path.join(output_frames, frame_name))
    if frame is not None:
        out.write(frame)

out.release()
print(f"Video saved to {output_video}")

In [ ]:
files.download(output_video)